In [ ]:
!pip install pyspark
!pip install pyspark[sql]

In [ ]:
import csv
import numpy as np
import math
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
import cv2
import matplotlib.pyplot as plt
from time import clock
from threading import Thread
from multiprocessing.pool import ThreadPool

In [ ]:

def loadData():

    img = cv2.imread('012345678910.png', 0)
    # plt.imshow(img)
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
    x = np.array(cells)
    # print(type(x))
    # print (x.shape)
    data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
    # print(data.shape)
    data=pd.DataFrame(data)
    # print(type(data))
    k = 0
    list = []
    for i in range(1, 5001):
        list.append(k)
        if i % 500 == 0:
            k = k + 1
    # print(list)
    list = np.array(list)
    list.reshape(5000, -1)
    list = pd.DataFrame(list)
    data["400"] = list

    spark = SparkSession.builder.getOrCreate()
    spark.conf.set("spark.sql.execution.arrow.enabled", "true")
    df = spark.createDataFrame(data)

    from pyspark.sql.functions import rand
    def getrows(df, rownums=None):
        return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: [x[0]])

    train=getrows(df, rownums=[x for x in range(5000) ]).collect()

    test=getrows(df, rownums=[x for x in range(25)]).collect()
    # thread = Thread(target=test)
    # thread.start()
    trainSet =[]
    for item in train:
      for x in item:
        trainSet.append (x)


    testSet =[]
    for item in test:
      for x in item:
        testSet.append (x)

    print ('luc dau :',type (testSet))
    # print (len (trainSet))
    trainSet=(np.array(trainSet)).tolist()
    testSet=(np.array(testSet)).tolist()
    print ('ve sau :',type (testSet))

    return  trainSet, testSet


In [ ]:
def calcDistancs(pointA, pointB, numOfFeature=400):

    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2

    return math.sqrt(tmp)




In [ ]:
def kNearestNeighbor(trainSet, point, k):

    distances = []
    for item in trainSet:
        distances.append({
            "label": item[-1],
            "value": calcDistancs(item, point)
        })
    distances.sort(key=lambda x: x["value"])
    labels = [item["label"] for item in distances]
    return labels[:k]


In [ ]:
def findMostOccur(arr):

    labels = set(arr)
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label


    return ans


In [ ]:
# if __name__ == "__main__":
def thucthi2():

    start=clock()
    trainSet, testSet = loadData()
    numOfRightAnwser = 0
    for item in testSet:
        knn = kNearestNeighbor(trainSet, item, 5)
        answer = findMostOccur(knn)
        numOfRightAnwser += item[-1] == answer
        print("label: {} -> predicted: {}".format(item[-1], answer))
    print("Accuracy", numOfRightAnwser/len(testSet))
    end=clock()
    duration=end-start
    print ("duration can tinh la :", duration)

In [ ]:
if __name__ == "__main__":
    pool = ThreadPool(5)
    parameters = [ 10, 20, 50]
    pool.map(lambda trees:thucthi2(),parameters)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


luc dau : <class 'list'>
ve sau :luc dau : <class 'list'>
 <class 'list'>
ve sau :luc dau : <class 'list'>
 <class 'list'>
ve sau : <class 'list'>
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0label: 0.0 -> predicted: 0.0

label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


label: 0.0 -> predicted: 0.0
Accuracy 1.0
duration can tinh la : 61.796006000000034


In [ ]:
from multiprocessing import Pool
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]


In [ ]:
bd_load=clock()
img = cv2.imread('012345678910.png', 0)
# plt.imshow(img)
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
x = np.array(cells)
# print(type(x))
# print (x.shape)
data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
# print(data.shape)
data=pd.DataFrame(data)
# print(type(data))
k = 0
list = []
for i in range(1, 5001):
    list.append(k)
    if i % 500 == 0:
        k = k + 1
# print(list)
list = np.array(list)
list.reshape(5000, -1)
list = pd.DataFrame(list)
data["400"] = list
print (data.shape)
data=np.array(data)
# spark = SparkSession.builder.getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# df = spark.createDataFrame(data)

# from pyspark.sql.functions import rand
# def getrows(df, rownums=None):
#     return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: [x[0]])

# train=getrows(df, rownums=[x for x in range(5000) ]).collect()

# test=getrows(df, rownums=[x for x in range(25)]).collect()
# # thread = Thread(target=test)
# # thread.start()
# trainSet =[]
# for item in train:
#   for x in item:
#     trainSet.append (x)


# testSet =[]
# for item in test:
#   for x in item:
#     testSet.append (x)

# print ('luc dau :',type (testSet))
# print (len (trainSet))
# trainSet=(np.array(trainSet)).tolist()
# testSet=(np.array(testSet)).tolist()
# print ('ve sau :',type (testSet))
# kt_load=clock()
# print ('thoi gian load data : ', kt_load - bd_load)
# return  trainSet, testSet

# gan nhan cho du lieu train
# k=np.arange(10)
# train_labels=np.repeat(k,250)[:,np.newaxis]
# # print(len(train_labels))
# #nhan dang
# knn=cv2.ml.KNearest_create()
# knn.train(trainSet,0,train_labels)
# kq1,kq2,kq3,kq4=knn.findNearest(testSet,5)
# print(kq2[805])
# print(len(kq2))
# print(train_labels[805])

(5000, 401)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [ ]:
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.getOrCreate()

In [ ]:
sc = SparkContext.getOrCreate()
# sqlContext = SQLContext(sc)
# data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
type (distData)

pyspark.rdd.RDD

In [ ]:
 len (distData.collect())

5000

In [ ]:
train =distData.collect()

In [ ]:
test =distData.take(25)

In [ ]:
lineLengths = distData.map(lambda s: len(s))
# totalLength = lineLengths.reduce(lambda a, b: a + b)
type (lineLengths)
print (lineLengths.take(10))


[401, 401, 401, 401, 401, 401, 401, 401, 401, 401]


In [ ]:
def loadData():

    img = cv2.imread('012345678910.png', 0)
    # plt.imshow(img)
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
    x = np.array(cells)
    # print(type(x))
    # print (x.shape)
    data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
    # print(data.shape)
    data=pd.DataFrame(data)
    # print(type(data))
    k = 0
    list = []
    for i in range(1, 5001):
        list.append(k)
        if i % 500 == 0:
            k = k + 1
    # print(list)
    list = np.array(list)
    list.reshape(5000, -1)
    list = pd.DataFrame(list)
    data["400"] = list
    data=np.array(data)
    spark = SparkSession.builder.getOrCreate()
    sc = SparkContext.getOrCreate()
    distData = sc.parallelize(data)
    trainSet =distData.collect()
    testSet =distData.take(26)


    # print ('luc dau :',type (testSet))
    # print (len (trainSet))
    trainSet=(np.array(trainSet)).tolist()
    testSet=(np.array(testSet)).tolist()
    # print ('ve sau :',type (testSet))

    return  trainSet, testSet

In [ ]:
def loadData1():

    img = cv2.imread('012345678910.png', 0)
    # plt.imshow(img)
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
    x = np.array(cells)
    # print(type(x))
    # print (x.shape)
    data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
    # print(data.shape)
    data=pd.DataFrame(data)
    # print(type(data))
    k = 0
    list = []
    for i in range(1, 5001):
        list.append(k)
        if i % 500 == 0:
            k = k + 1
    # print(list)
    list = np.array(list)
    list.reshape(5000, -1)
    list = pd.DataFrame(list)
    data["400"] = list
    data=np.array(data)
    spark = SparkSession.builder.getOrCreate()
    sc = SparkContext.getOrCreate()
    distData = sc.parallelize(data)
    trainSet1 =distData.collect()
    testSet1 =distData.take(13)
    # spark = SparkSession.builder.getOrCreate()
    # spark.conf.set("spark.sql.execution.arrow.enabled", "true")
    # df = spark.createDataFrame(data)

    # from pyspark.sql.functions import rand
    # def getrows(df, rownums=None):
    #     return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: [x[0]])

    # train=getrows(df, rownums=[x for x in range(5000) ]).collect()

    # test=getrows(df, rownums=[x for x in range(25)]).collect()
    # # thread = Thread(target=test)
    # # thread.start()
    # trainSet =[]
    # for item in train:
    #   for x in item:
    #     trainSet.append (x)


    # testSet =[]
    # for item in test:
    #   for x in item:
    #     testSet.append (x)

    # print ('luc dau :',type (testSet))
    # print (len (trainSet))
    trainSet1=(np.array(trainSet)).tolist()
    testSet1=(np.array(testSet)).tolist()
    # print ('ve sau :',type (testSet))

    return trainSet1, testSet1

In [ ]:
def calcDistancs(pointA, pointB, numOfFeature=400):

    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2

    return math.sqrt(tmp)

In [ ]:
def calcDistancs1(pointA, pointB, numOfFeature=400):

    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2

    return math.sqrt(tmp)

In [ ]:
def kNearestNeighbor(trainSet, point, k):

    distances = []
    for item in trainSet:
        distances.append({
            "label": item[-1],
            "value": calcDistancs(item, point)
        })
    distances.sort(key=lambda x: x["value"])
    labels = [item["label"] for item in distances]
    return labels[:k]

In [ ]:
def kNearestNeighbor1(trainSet, point, k):

    distances = []
    for item in trainSet:
        distances.append({
            "label": item[-1],
            "value": calcDistancs(item, point)
        })
    distances.sort(key=lambda x: x["value"])
    labels = [item["label"] for item in distances]
    return labels[:k]

In [ ]:
def findMostOccur(arr):

    labels = set(arr)
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label


    return ans

In [ ]:
def findMostOccur1(arr):

    labels = set(arr)
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label


    return ans

In [ ]:
# if __name__ == "__main__":
def thucthi():
    start=clock()
    trainSet, testSet = loadData()
    numOfRightAnwser = 0
    for item in testSet:
        knn = kNearestNeighbor(trainSet, item, 5)
        answer = findMostOccur(knn)
        numOfRightAnwser += item[-1] == answer
        # print("label: {} -> predicted: {}".format(item[-1], answer))
    print("Accuracy", numOfRightAnwser/len(testSet))
    end=clock()
    duration=end-start
    print ("duration can tinh la :", duration)

In [ ]:
def thucthi1(x ):

    trainSet, testSet=x
    numOfRightAnwser = 0
    for item in testSet:
        knn = kNearestNeighbor1(trainSet, item, 5)
        answer = findMostOccur1(knn)
        numOfRightAnwser += item[-1] == answer
        print("label: {} -> predicted: {}".format(item[-1], answer))
    print("Accuracy", numOfRightAnwser/len(testSet))


In [ ]:
from multiprocessing import Pool
if __name__ == "__main__":
    start1=clock()
    with Pool(15) as p:
        print(p.map(thucthi1, [loadData(),loadData1()]))
    end1=clock()
    duration1=end1-start1
    print ("duration can tinh la :", duration)